In [ ]:
from src.database_utils.db_values.preprocess import make_db_lsh
from src.database_utils.db_catalog.preprocess import make_db_context_vec_db

In [ ]:
for db_id in ['debit_card_specializing', 'financial', 'regional_sales']:
    make_db_lsh(
        f'/home/chenjie/projects/chase-sql-on-finch/data/bird/{db_id}', 
        signature_size=100, 
        n_gram=3, 
        threshold=0.01,
        verbose=True)
    make_db_context_vec_db(
        f'/home/chenjie/projects/chase-sql-on-finch/data/bird/{db_id}',
        use_value_description=True)

In [ ]:

make_db_lsh(
        '/home/chenjie/projects/chase-sql-on-finch/data/bool_sql/accounting', 
        signature_size=100, 
        n_gram=3, 
        threshold=0.01,
        verbose=True)
make_db_context_vec_db(
    '/home/chenjie/projects/chase-sql-on-finch/data/bool_sql/accounting', 
    use_value_description=False)

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path

conn = sqlite3.connect('/home/chenjie/projects/chase-sql-on-finch/data/bool_sql/accounting/accounting.sqlite')
cursor = conn.cursor()

# Get all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

metadata = []

for table in tables:
    table_name = table[0]
    
    # Get column information for each table
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    
    for column in columns:
        # column tuple: (cid, name, type, notnull, dflt_value, pk)
        column_name = column[1]
        column_type = column[2]
        
        metadata.append({
            'table_name': table_name,
            'column_name': column_name,
            'original_column_name': column_name
        })

conn.close()

# Save to CSV
df = pd.DataFrame(metadata)
for table_name in df.table_name.unique():
    table_df = df[df.table_name == table_name].copy()
    output_path = Path('/home/chenjie/projects/chase-sql-on-finch/data/bool_sql/accounting/database_description') / f'{table_name}.csv'
    table_df.to_csv(output_path, index=False)

In [ ]:
df